In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## What are experts?
- Experts are group of neural network also called feed forward layer in dense layer. Instead of using dense layer, we divide neural network in many of small type(same input, hidden and output layers).

## What is GeLU?
- Stands for Gaussian Error Linear Unit
- uses Cumulative Distributive function for small negatives
- better then relu as it doesn't neglate negatives

In [ ]:
class Expert(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super().__init__()
    self.net=nn.Sequential(
        nn.Linear(input_dim, hidden_dim),
        nn.GeLU(),
        nn.Linear(hidden_dim, output_dim)

    )
